In [86]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import random

In [87]:
# The Original Data (this is just one data point)
df = pd.read_csv('/Users/charbel/Desktop/04-12-2020.csv')
df

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3563,93,NaN,3470,1.0,75.988020,21583.0,437.0,2.610160,84000001,USA,460.300152,12.264945
1,Alaska,US,2020-04-12 23:18:15,61.3707,-152.4044,272,8,66.0,264,2.0,45.504049,8038.0,31.0,2.941176,84000002,USA,1344.711576,11.397059
2,Arizona,US,2020-04-12 23:18:15,33.7298,-111.4312,3542,115,NaN,3427,4.0,48.662422,42109.0,NaN,3.246753,84000004,USA,578.522286,NaN
3,Arkansas,US,2020-04-12 23:18:15,34.9697,-92.3731,1280,27,367.0,1253,5.0,49.439423,19722.0,130.0,2.109375,84000005,USA,761.753354,10.156250
4,California,US,2020-04-12 23:18:15,36.1162,-119.6816,22795,640,NaN,22155,6.0,58.137726,190328.0,5234.0,2.812020,84000006,USA,485.423868,22.961176
5,Colorado,US,2020-04-12 23:18:15,39.0598,-105.3111,7307,289,NaN,7018,8.0,128.943729,34873.0,1376.0,3.955112,84000008,USA,615.389991,18.831258
6,Connecticut,US,2020-04-12 23:18:15,41.5978,-72.7554,12035,554,NaN,11481,9.0,337.560483,41220.0,1654.0,4.603241,84000009,USA,1156.148159,13.743249
7,Delaware,US,2020-04-12 23:18:15,39.3185,-75.5071,1625,35,191.0,1590,10.0,166.878217,11103.0,190.0,2.153846,84000010,USA,1140.214672,11.692308
8,Diamond Princess,US,2020-04-12 23:18:15,NaN,NaN,49,0,0.0,49,888.0,NaN,NaN,NaN,0.000000,84088888,USA,NaN,NaN
9,District of Columbia,US,2020-04-12 23:18:15,38.8974,-77.0268,1875,50,493.0,1825,11.0,265.675190,10640.0,NaN,2.666667,84000011,USA,1507.618148,NaN


In [88]:
# Dropping unecessary rows and replacing NaN with column median
df = pd.read_csv('/Users/charbel/Desktop/04-12-2020.csv')
df = df.drop(["Lat", "Long_", "Last_Update", "Province_State", "Country_Region", "UID", "ISO3", "FIPS"], 1)
df = df.drop([57])
print(df.median())
df = df.fillna(df.median())
df

Confirmed                2136.500000
Deaths                     76.000000
Recovered                 235.500000
Active                   2079.500000
Incident_Rate              71.401808
People_Tested           26651.500000
People_Hospitalized       437.000000
Mortality_Rate              2.812020
Testing_Rate              793.614360
Hospitalization_Rate       12.662338
dtype: float64


,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
0,3563,93,235.5,3470,75.988020,21583.0,437.0,2.610160,460.300152,12.264945
1,272,8,66.0,264,45.504049,8038.0,31.0,2.941176,1344.711576,11.397059
2,3542,115,235.5,3427,48.662422,42109.0,437.0,3.246753,578.522286,12.662338
3,1280,27,367.0,1253,49.439423,19722.0,130.0,2.109375,761.753354,10.156250
4,22795,640,235.5,22155,58.137726,190328.0,5234.0,2.812020,485.423868,22.961176
5,7307,289,235.5,7018,128.943729,34873.0,1376.0,3.955112,615.389991,18.831258
6,12035,554,235.5,11481,337.560483,41220.0,1654.0,4.603241,1156.148159,13.743249
7,1625,35,191.0,1590,166.878217,11103.0,190.0,2.153846,1140.214672,11.692308
8,49,0,0.0,49,71.401808,26651.5,437.0,0.000000,793.614360,12.662338
9,1875,50,493.0,1825,265.675190,10640.0,437.0,2.666667,1507.618148,12.662338


In [89]:
# (# of days, #of states, #of features)
x = df.to_numpy()
print(x.shape)
x = np.expand_dims(x, axis=0)
print(x.shape)

(58, 10)
(1, 58, 10)


In [90]:
class RNN(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.rnn = nn.GRU(input_size=58*10,
                          hidden_size=hidden_size,
                          batch_first=True)
        self.proj = nn.Linear(in_features=hidden_size,
                              out_features=58)
    
    def forward(self, inp):
        X = inp.reshape([-1,580]).unsqueeze(dim=0)
        out, last_hidden = self.rnn(X)
        out_seq = self.proj(out)
        return out_seq

In [91]:
rnn = RNN(2)
y = rnn(torch.Tensor(x)).squeeze()
y.shape

torch.Size([58])

In [92]:
y

tensor([-0.1060,  0.0241, -0.2102,  0.1738,  0.4449,  0.3437,  0.1160, -0.0032,
        -0.0656, -0.3629, -0.3043,  0.3036, -0.3846, -0.1220, -0.4275,  0.0440,
         0.1163, -0.3170, -0.2479,  0.0667,  0.6086, -0.2773, -0.0230, -0.5635,
         0.5491, -0.0377, -0.5974,  0.2596,  0.2962, -0.6569, -0.4775, -0.0971,
         0.0377,  0.4295, -0.1301, -0.5055,  0.1697, -0.3698, -0.3038,  0.3649,
         0.5843,  0.3876, -0.4641,  0.4349, -0.1934,  0.2439,  0.0739,  0.2796,
         0.1586, -0.1285,  0.6788,  0.0549, -0.3404, -0.5878, -0.6272, -0.2146,
        -0.5958,  0.0806], grad_fn=<SqueezeBackward0>)